# Progetto Foundation of Computer Science

#### Università di Milano Bicocca, Andrea Marinoni 799690

Viene importata la libreria **pandas** e la libreria **re**

In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

I dataset vengono importati, si presta attenzione al separatore delle colonne

In [2]:
books=pd.read_csv(r'Books.csv', sep=';', encoding='latin1')
books_rating=pd.read_csv(r'Book-Ratings.csv', sep=';', encoding='latin1')                        
goodbooks=pd.read_csv(r'goodbooks.csv', sep=',', encoding='latin1')
goodbooks_rating=pd.read_csv(r'goodbooks-ratings.csv', sep=',', encoding='latin1')
users=pd.read_csv(r'Users.csv', sep = ';', encoding='latin1')

**1. Normalize the location field of Users dataset, splitting into city, region, country.**

Si normalizza il testo del campo Location

In [3]:
users['Location'] = users['Location'].apply(str)
users['Location'] = users['Location'].str.lower()
users['Location'] = users['Location'].str.replace(r'\d+', '', regex=True)

Il campo Location viene splittato per ottenere city, region, country

In [4]:
df= users.Location.str.split(',',expand=True,)       #l'ideale sarebbe stato ottenere solo 3 colonne, 
                                                     #vuol dire che i dati non sono stati regisrati correttamente
df.head(2)

,0,1,2,3,4,5,6,7,8
0,nyc,new york,usa,None,None,None,None,None,None
1,stockton,california,usa,None,None,None,None,None,None


In [5]:
df[3].notnull().sum()                               #Si quantificano grosso modo le righe con campo Location non corretto

1508

In [6]:
users['Location'] = users['Location'].str.replace(r'\b(\w+)(,+\s+\1)+\b', r'\1')     #Si prova a risolvere il problema, eliminando i duplicati

In [7]:
df= users.Location.str.split(',',expand=True,)     #Si riesegue lo split
df[3].notnull().sum()                              #e si riquantificano gli errori, che sono minori

1281

In [8]:
df = df[df[3].isna()]                              #Vengono eliminate le rows con errori

Le nuove colonne city, region e country vengono importate nel dataset users

In [9]:
users[['city', 'region', 'country']]=df[[0,1,2]]
users=users[['User-ID','Age','city','region','country']]       #Nelle istanze con il dato Location raccolto male ci saranno dei null value, si
users.head(2)                                                  #è deciso di operare in questa maniera e non cancellando l'intera riga, 
                                                               #in modo da non perdere le altre indicazioni presenti

,User-ID,Age,city,region,country
0,1,NaN,nyc,new york,usa
1,2,18.0,stockton,california,usa


**2. For each book in the Books dataset, compute its average rating.**

I dataset books e books_rating vengono uniti, tramite la chiave ISBN

In [10]:
books_finale=pd.merge(books, books_rating, left_on='ISBN', right_on='ISBN', how='left')

Si esegue una groupby per l'attributo ISBN e su di essa si esegue una media, cosi da ottenere il rating medio per ogni libro

In [11]:
avg_books_rating=books_finale.groupby('ISBN').mean()['Book-Rating']

**3. For each book in the GoodBooks dataset, compute its average rating.**

I dataset goodbooks e goodbooks_rating vengono uniti tramite la chiave book_id 

In [12]:
goodbooks_finale=pd.merge(goodbooks, goodbooks_rating, left_on='book_id', right_on='book_id', how='left')

Si esegue una groupby per l'attributo book_id e su di essa si esegue una media, cosi da ottenere il rating medio per ogni libro

In [13]:
avg_goodbooks_rating=goodbooks_finale.groupby('book_id').mean()['rating']

**4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset merged books. The books that have not been merged together will not appear in merged books.**

Si esegue una groupby contemporaneamente per book title, author and publisher, si imposta il filtro con len > 1 per escludere tutti i libri
che non sono stati raggruppati

In [14]:
merged_books=books.groupby(['Book-Title','Book-Author','Publisher']).filter(lambda x: len(x) > 1)

**5. For each book in merged books compute its average rating.**

I dataset merged_books e books_rating vengono uniti tramite la chiave ISBN

In [15]:
merged_books_finale=pd.merge(merged_books, books_rating, left_on='ISBN', right_on='ISBN')

Si esegue una groupby per l'attributo ISBN e su di essa si esegue una media, cosi da ottenere il rating medio per ogni libro raggruppato

In [16]:
avg_rat_m_b_finale=merged_books_finale.groupby('Book-Title')['Book-Rating'].mean()

**6. For each book in merged books compute the minimum and maximum of the average ratings over all corresponding books in the books dataset.**

Vengono eseguiti il minimo ed il massimo, e poi aggiunti con una concatenazione al dataset merged_rating originale

In [17]:
max_rat_m_b_finale=merged_books_finale.groupby('Book-Title')['Book-Rating'].max()
min_rat_m_b_finale=merged_books_finale.groupby('Book-Title')['Book-Rating'].min()
merged_rating=pd.concat([avg_rat_m_b_finale, max_rat_m_b_finale, min_rat_m_b_finale], axis=1)
merged_rating.columns = ['AVG', 'MAX', 'MIN']

**7. For each book in goodbooks, compute the list of its authors. Assuming that the number of reviews with a text (column work_text_reviews_count) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the shared number of reviews with a text.**

Il testo del campo authors in goodbooks viene normalizzato

In [18]:
goodbooks['authors'] = goodbooks['authors'].apply(str)
goodbooks['authors'] = goodbooks['authors'].str.lower()
goodbooks['authors'] = goodbooks['authors'].str.replace(r'\d+', '', regex=True)

Il campo authors viene splittato per ottenere tutti gli attori in colonne separate

In [19]:
authors= goodbooks.authors.str.split(',',expand=True,)
authors=pd.concat([goodbooks['title'],authors,goodbooks['work_text_reviews_count']], axis=1)

Viene creata la funzione found_authors in grado di trovare tutti gli attori presenti e costruirne una lista

In [20]:
def found_authors(df):
    autori=[]
    for a in range(0,5):
        for i in df[a]:
            if i != None and i not in autori:
                autori.append(i)
    return(autori)

In [21]:
autori=found_authors(authors)

Viene creata la funzione conta, con l'obiettivo di contare il numero di autori per titolo di film

In [22]:
def conta (df):
    conteggio=[]
    for i in range(0,99):
        count=0
        for e in range(0,5):
                if df[e].iloc[i]!=None:
                    count=count+1
        conteggio.append(count)
    return(conteggio)

In [23]:
conteggio=conta(authors)
authors['conteggio']=conteggio

Si calcolano le work_text_reviews_count ad autore per libro

In [24]:
authors['reviews_per_author']=authors['work_text_reviews_count']/authors['conteggio']

Viene creata la funzione reviews, con l'obiettivo di calcolare la somma totale delle work_text_reviews per authors

In [25]:
def reviews(df):
    reviews=[]
    for a in autori:
        conta=0
        for i in range(0,99):
            for e in range(0,5):
                if a == df[e].iloc[i]:
                    conta=conta+df['reviews_per_author'].iloc[i]
        reviews.append(conta)
    return(reviews)

In [26]:
shared_number_of_reviews_with_a_text=reviews(authors)

In [27]:
authors_final = pd.DataFrame(list(zip(autori, shared_number_of_reviews_with_a_text)),columns=['autori','shared_number_of_reviews_with_a_text'])

**8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.**

Si corregge il campo isbn in goodbooks nel quale erano stati rimossi degli zero in fase di realizzazione, e si normalizza il testo
del campo Book-Author in books

In [28]:
goodbooks['isbn'] = goodbooks['isbn'].apply(str)
goodbooks['isbn'] = goodbooks['isbn'].apply(lambda x: x.zfill(10))
books['Book-Author'] = books['Book-Author'].apply(str)
books['Book-Author'] = books['Book-Author'].str.lower()
books['Book-Author'] = books['Book-Author'].str.replace(r'\d+', '', regex=True)

Vengono uniti il dataset authors_final e books

In [29]:
libri=pd.merge(authors_final, books, left_on='autori', right_on='Book-Author')

Per ogni anno si visualizza l'autore con più alto numero di reviews with a text

In [30]:
result=libri.groupby('Year-Of-Publication').idxmax()['shared_number_of_reviews_with_a_text']

In [31]:
autforyear=libri[['Year-Of-Publication','autori','shared_number_of_reviews_with_a_text']].iloc[result]

**9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets**

Si corregge il campo isbn in goodbooks_finale nel quale erano stati rimossi degli zero in fase di realizzazione

In [32]:
goodbooks_finale['isbn'] = goodbooks_finale['isbn'].apply(str)
goodbooks_finale['isbn'] =goodbooks_finale['isbn'].apply(lambda x: x.zfill(10))

I dataset books_finale e goodbooks_finale sono uniti tramite i rispettivi attributi ISBN e isbn

In [33]:
mergedbooks2=pd.merge(books_finale, goodbooks_finale, left_on='ISBN', right_on='isbn')

Si trova la media dei rating

In [34]:
avg_mergedbooks2=mergedbooks2.groupby('ISBN').mean()[['rating','Book-Rating']]

Si calcola la differenza tra rating

In [35]:
avg_mergedbooks2['diff_rating']=avg_mergedbooks2['Book-Rating']-avg_mergedbooks2['rating']
avg_mergedbooks2 = avg_mergedbooks2.set_axis(['books_rating', 'goodbooks_rating', 'diff_rating'], axis=1, inplace=False)
avg_mergedbooks2 = avg_mergedbooks2.dropna()             #vengono eliminate le righe inutili perchè avevano una mancanza in uno dei due rating

**10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.**

Il dataset users viene diviso in tanti dataset in base alle fasce d'età

In [36]:
users_nan=users[users['Age'].isnull()]
users_0_14=users[users['Age']<=14]
users_15_24=users[users['Age']<=24]
users_25_34=users[users['Age']<=34]
users_35_44=users[users['Age']<=44]
users_45_54=users[users['Age']<=54]
users_55_64=users[users['Age']<=64]
users_65_74=users[users['Age']<=74]
users_75_84=users[users['Age']<=84]
users_85_94=users[users['Age']<=94]
users_95_104=users[users['Age']<=104]

**11. Find the books that appear only in the goodbooks datasets.**

Si cercano i libri di goodbooks in books e si pone la condizione false

In [37]:
goodbooks_not_in_books = goodbooks[goodbooks.isbn.isin(books.ISBN) == False]

**12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?**

Si è utilizzato size per calcolare il numero di volte che si è raggruppato e poi si è trovato il massimo

In [38]:
df12=books.groupby(['Book-Title','Book-Author']).size().reset_index(name='count')
book_max=df12[df12['count']==df12['count'].max()]

**13. Find the author with the highest average rating according to the goodbooks datasets**

Si è aggionato il dataset goodbooks_finale con gli autori scritti normalizzati

In [39]:
goodbooks_finale=pd.merge(goodbooks, goodbooks_rating, left_on='book_id', right_on='book_id', how='left')

Per comodità si è deciso di creare un'altro dataset degli autori, così da inserire i rating

In [40]:
authors2= goodbooks_finale.authors.str.split(',',expand=True,)
authors2=pd.concat([goodbooks_finale['title'],authors2,goodbooks_finale['rating']], axis=1)

In [41]:
autori2=found_authors(authors2)

Si è creata la funzione voto_medio per creare il voto medio del rating per autore

In [42]:
def voto_medio(df):
    votazione=[]
    conteggio=[]
    for a in autori2:
        voto=0
        conta=0
        for i in range(0,102):
            for e in range(0,5):
                if a == df[e].iloc[i]:
                    voto=voto+df['rating'].iloc[i]
                    conta=conta+1
        votazione.append(voto)
        conteggio.append(conta)
    voto_medio=[a/b for a,b in zip(votazione, conteggio)]
        
    return(voto_medio)

In [43]:
votazione=voto_medio(authors2)

In [44]:
authors_final2 = pd.DataFrame(list(zip(autori2, votazione)),columns=['autori','voto_medio'])

E'stato trovato l'autore con voto medio maggiore

In [45]:
authors_final2.iloc[authors_final2['voto_medio'].idxmax()]

autori        paulo coelho
voto_medio               5
Name: 23, dtype: object